In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W20` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W21`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W22` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W23` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W24` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W25` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W26` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W27` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W28` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W29`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W30`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W31`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W32`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.head()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
0,2020-05-21 08:43:00+00:00,2020-05-21 08:45:00+00:00,TBD,None,N4,None,None,KL19,None,Thursday Morning,Dummy,0,1.73,2020-22
1,2020-05-24 08:52:00+00:00,2020-05-24 08:56:00+00:00,Break,None,J4,move to other line,None,KL15,None,Sunday Morning,Dummy,0,3.15,2020-22
2,2020-07-16 09:28:11+00:00,2020-07-16 09:28:29+00:00,Preparation/Changeover,None,F4,None,None,KL10,None,Thursday Morning,Temp,0,0.30,2020-30
3,2020-08-01 07:39:17+00:00,2020-08-01 09:52:59+00:00,Factory Closed,None,C4,None,None,KL3,None,Saturday Morning,Leon,0,133.70,2020-32
4,2020-06-06 16:54:00+00:00,2020-06-06 16:55:00+00:00,TBD,None,J2,None,None,KL6,None,Saturday Afternoon,William,0,1.32,2020-24


In [5]:
data_in_use.shape

(92833, 14)